**Goal of this notebook is to expand the high_diamond_ranked_10min.csv dataset with the information about player compositions and personal player information using Cassiopeia API**

In [26]:
!pip install cassiopeia

In [42]:
from cassiopeia.core import Summoner, MatchHistory, Match
from cassiopeia import Champion, ChampionMastery
from cassiopeia import Queue, Patch
import pandas as pd
import random
from math import ceil,floor

import cassiopeia as cass

cass.set_riot_api_key("Your key")  # This overrides the value set in your configuration/settings.
cass.set_default_region("NA")


In [40]:
df = pd.read_csv('/content/drive/My Drive/LolAnalytics/high_diamond_ranked_10min.csv')

In [41]:
def getmatch(record):
  ID = int(record['gameId'])
  servers = [region.value for region in list((cass.Region))]
  for server in servers: # This is necessary because riot api wants us to specify region and our data doesnt include that
      try:
        match1 = cass.get_match(id = ID , region = server)
        match1.red_team.participants
      except Exception:
        pass
      else:
        print('match found')
        return match1

In [30]:
# def get_champion_compositions(match):
  
#   def turn_dict(list_of_tuples):
#     return {f'Summoner {i}': my_tuple for i, my_tuplee in enumerate(list_of_tuples)}
  
#   blue_team_composition = [(participant.role.value,participant.champion.id,participant.champion.name) for participant in list(match.blue_team.participants)]
#   red_team_composition = [(participant.role.value,participant.champion.id,participant.champion.name) for participant in list(match.red_team.participants)]
#   return turn_dict(blue_team_composition), turn_dict(red_team_composition)

In [31]:
def get_champion_compositions(match):

      blue_team_composition = {f'Blue_champ_{i+1}':participant.champion.name for i,participant in enumerate(list(match.blue_team.participants))}
      red_team_composition = {f'Red_champ_{i+1}':participant.champion.name for i,participant in enumerate(list(match.red_team.participants))}
      return blue_team_composition, red_team_composition

In [32]:
def get_lowest_team_account_level(match): # pretty good predictor of having a smurf in team if < 50-70
  
  lowest_lvl_blue = 500
  region1 = match.region
  
  for participant in match.blue_team.participants:
    
    player = Summoner(name=participant.summoner.name, region=region1)
    if isinstance(player, Summoner) == False:
      continue
    if player.level <= lowest_lvl_blue:
      
      lowest_lvl_blue = player.level
    

  lowest_lvl_red = 500
  
  for participant in match.red_team.participants:
    
    player = Summoner(name=participant.summoner.name, region=region1)
    if isinstance(player, Summoner) == False:
      continue
    if player.level <= lowest_lvl_red:
      
      lowest_lvl_red = player.level
    

  
  return {'lowest_lvl_red':lowest_lvl_red, 'lowest_lvl_blue':lowest_lvl_blue}

In [33]:
def get_average_team_maestry(match):  
  
  blue_team_sum = 0
  region1 = match.region
  
  for participant in match.blue_team.participants:
    
    player = Summoner(name=participant.summoner.name, region=region1)
    champ = Champion(name=participant.champion.name, id = participant.champion.id, region =  region1)
    cm = cass.get_champion_mastery(champion=champ, summoner=player, region =  region1)
    blue_team_sum += cm.points

  red_team_sum = 0
  
  for participant in match.red_team.participants:
    
    player = Summoner(name=participant.summoner.name, region=region1)
    champ = Champion(name=participant.champion.name, id = participant.champion.id, region = region1)
    cm = cass.get_champion_mastery(champion=champ, summoner=player, region =  region1)
    red_team_sum += cm.points
  
  return {'Blue_avg_mastery': ceil(blue_team_sum/5),'Red_avg_mastery':ceil(red_team_sum/5)}

In [34]:
def add_information_to_row(row,list_of_dicts):

  for dictionary in list_of_dicts:

    for item in dictionary.items():
      row[str(item[0])] = item[1]
  
  return row

In [35]:
def update_row(row):
  
  match = getmatch(row)                       # can optimize by changing the order of servers into EUW -> EUNE -> NA-> KOREA -> REST
  #a = get_average_team_maestry(match)         # sends 10 requests per iteration
  b = get_lowest_team_account_level(match)    # 10 requests per iteration
  c,d = get_champion_compositions(match)      # 1 request per iteration
  add_information_to_row(row,[b,c,d])
  return row

In [36]:
def update_dataframe(df):
  new_df = pd.DataFrame()
  
  for i in range(0,len(df)):
    row = df.iloc[i]
    try:
      update_row(row)
    except Exception:
      pass
    else:
      new_df = new_df.append(update_row(row),ignore_index = True)

    print(f'{i+1} of {len(df)} iterations complete')
  return new_df

Due to the riot API request limit of 50/min i can't update entire dataset with all the features i wanted, so i'll instead update a subset of the data, feed it to a model and use its output as a feature to a meta model. Also i didnt use team mastery as a feature because it roughly doubles the time for each iteration and it isnt really a good predictor of ones mastery over a champion(ironically). 

In [43]:
sample = df.sample(200)
df_with_new_features = update_dataframe(sample)

Making call: https://br1.api.riotgames.com/lol/match/v4/matches/4515839148
Making call: https://eun1.api.riotgames.com/lol/match/v4/matches/4515839148
Making call: https://euw1.api.riotgames.com/lol/match/v4/matches/4515839148
match found
Making call: https://euw1.api.riotgames.com/lol/summoner/v4/summoners/YTzZxQuSlcu6d13BB0maO-9wpzbaZSQCTZJJqOyniOBfzPA
Making call: https://euw1.api.riotgames.com/lol/summoner/v4/summoners/396CUX8OXY8YLs9c32bHZihteJZBKduZHAiW0-D9F67QSdI


/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  
/usr/local/lib/python3.6/dist-packages/pandas/core/indexing.py:671: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_with_indexer(indexer, value)


Making call: https://br1.api.riotgames.com/lol/match/v4/matches/4515839148
Making call: https://eun1.api.riotgames.com/lol/match/v4/matches/4515839148
match found
1 of 200 iterations complete
Making call: https://br1.api.riotgames.com/lol/match/v4/matches/4527239272
Making call: https://eun1.api.riotgames.com/lol/match/v4/matches/4527239272
Making call: https://euw1.api.riotgames.com/lol/match/v4/matches/4527239272
match found
Making call: https://euw1.api.riotgames.com/lol/summoner/v4/summoners/Ev1YAMtfb87vEVJHenwC2pK7hFUdeJ7u__Fp_M7LwnYGb2ZP
Making call: https://euw1.api.riotgames.com/lol/summoner/v4/summoners/_Te2AkPqKfO5-u9CQ5cxAiFH1CUoSBRTS-kIjbBQiP4u64I
Making call: https://euw1.api.riotgames.com/lol/summoner/v4/summoners/utRLLQiXZoUmNmJVLWLMRI41bwQ1Szoum495E0Q-XylcDpQ
Making call: https://euw1.api.riotgames.com/lol/summoner/v4/summoners/kygvYiohmDitgEc7DtM2Y7IC7GWrvrMw02Q2m1Kb0B6ymuY
Making call: https://euw1.api.riotgames.com/lol/summoner/v4/summoners/Mjd9j0lq-CVlcGfM8XJL3H3q27

In [45]:
df_with_new_features.to_csv('/content/drive/My Drive/LolAnalytics/high_diamond_ranked_10min_with_added_features.csv')